In [1]:
from bs4 import BeautifulSoup
import os

In [2]:
filepath1 = "C:/Users/rschl/Desktop/Image Labeling/annotated data/AmanitaPhalloides/AmanitaPhalloides_1.xml"
filepath2 = "C:/Users/rschl/Desktop/Image Labeling/annotated data/AmanitaPhalloides/AmanitaPhalloides_0.xml"

In [3]:
def parseXML(filepath):
    infile = open(filepath,'r')
    content = infile.read()
    soup = BeautifulSoup(content,'xml')
    width = int(soup.find('width').text)
    height = int(soup.find('height').text)
    xmins = soup.findAll('xmin')
    ymins = soup.findAll('ymin')
    xmax = soup.findAll('xmax')
    ymax = soup.findAll('ymax')
    boxes = []
    for i in range(len(xmins)):
        box = []
        box.append(int(xmins[i].contents[0]))
        box.append(int(ymins[i].contents[0]))
        box.append(int(xmax[i].contents[0]))
        box.append(int(ymax[i].contents[0]))
        boxes.append(box)

    return width, height, boxes


In [4]:
def convert_coord(pascal):
    width = pascal[0]
    height = pascal[1]
    
    list_boxes = pascal[2]

    yolo = []

    for i in range(len(list_boxes)):
        yolo_box = []
        
        xmin = list_boxes[i][0]
        ymin = list_boxes[i][1]
        xmax = list_boxes[i][2]
        ymax = list_boxes[i][3]

        yolo_x = ((xmax-xmin)/2 + xmin)/width
        yolo_y = ((ymax-ymin)/2 + ymin)/height

        yolo_width = round((xmax-xmin)/width,6)
        yolo_height = round((ymax-ymin)/height,6)

        yolo_box.append(format(yolo_x,'.6f'))
        yolo_box.append(format(yolo_y,'.6f'))
        yolo_box.append(format(yolo_width,'.6f'))
        yolo_box.append(format(yolo_height,'.6f'))

        yolo.append(yolo_box)

    return yolo


In [5]:
def convert_yolo_list(yolo_box):
    yolo_complete = ""
    for i in range(len(yolo_box)):    
        curr_box = yolo_box[i]
        yolo_str = "0 "
        for i in range(len(curr_box)):
            yolo_str += curr_box[i]
            yolo_str += " "
        
        yolo_str = yolo_str[:-1]
        yolo_str += "\n"
        yolo_complete += yolo_str


    return yolo_complete

In [6]:
data = [
    'annotated_data\Amanita Pantherina',
    'annotated_data\AmanitaPantherina',
    'annotated_data\AmanitaPhalloides'
    'annotated_data\Conocybe'
    'annotated_data\CoprinopsisAtramentaria'
]

In [7]:
def combined_conversion(file, destination):
    try:
        f = open(destination,'w')
    except:
        print("Something went wrong opening the destination folder")
    try:
        f.write(convert_yolo_list(convert_coord(parseXML(file))))
    except:
        print("Something went wrong writing the text file")

In [8]:
def convert_directory(dir):
    file_list = os.listdir(dir)

    if not os.path.exists(dir + '\yolo'):
        os.makedirs(dir + '\yolo')

    for i in range(len(file_list)):
        xml_path = dir + '\\' + file_list[i]
        destination = dir + '\yolo' + '\\' + file_list[i][:-3] + "txt"
        print("Converting", xml_path, "to destination", destination)

        combined_conversion(xml_path,destination)


        

In [9]:
for i in range(len(data)):
    convert_directory(data[i])

Converting annotated_data\Amanita Pantherina\AmanitaPantherina_1.xml to destination annotated_data\Amanita Pantherina\yolo\AmanitaPantherina_1.txt
Converting annotated_data\Amanita Pantherina\AmanitaPantherina_10.xml to destination annotated_data\Amanita Pantherina\yolo\AmanitaPantherina_10.txt
Converting annotated_data\Amanita Pantherina\AmanitaPantherina_100.xml to destination annotated_data\Amanita Pantherina\yolo\AmanitaPantherina_100.txt
Converting annotated_data\Amanita Pantherina\AmanitaPantherina_101.xml to destination annotated_data\Amanita Pantherina\yolo\AmanitaPantherina_101.txt
Converting annotated_data\Amanita Pantherina\AmanitaPantherina_102.xml to destination annotated_data\Amanita Pantherina\yolo\AmanitaPantherina_102.txt
Converting annotated_data\Amanita Pantherina\AmanitaPantherina_103.xml to destination annotated_data\Amanita Pantherina\yolo\AmanitaPantherina_103.txt
Converting annotated_data\Amanita Pantherina\AmanitaPantherina_104.xml to destination annotated_data

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'annotated_data\\AmanitaPhalloidesannotated_data\\Conocybeannotated_data\\CoprinopsisAtramentaria'